# Ensemble

## Descrição do exercício
1. Avalie o desempenho dos algoritmos Ensemble com DecisionTree, RandomForest e Ada Boosting para o dataset "Wisconsin Diagnostic Breast Cancer (WDBC)". Separe os dados em treino (80%) e teste (20%).
1. Realize o escalonamento das features usando o StandardScaler.
1. Use um valor constante para o parâmetro random_state e ajuste o modelo com as seguintes combinações de hiper-parâmetros para RandomForest e Ada Boosting.
1. Utilize o método Grid-Search com 5-fold cross-validation para ajustar os hiper-parametros do modelo. Vrie os hiper parametros em: learning_rate: 0.1, 0.05, 0.01 ,0.5; n_estimators: 50, 100, 200; max_depth: 3, 5, 7
1. Retreine o modelo com a melhor combinação de parametros usando o dataset de treino por completo e obtenha as métricas de acurácia, recall, F1-score e precision. Apresente o valor das médias e de cada classe individualmete. Discuta os resultados, avaliando o comportamento de cada modelo em função das métricas.
1. Mostre a importância das features de acordo com o melhor modelo de classificação e o melhor modelo encontrado dentre os 3 usados nesta lista.


### Importações Gerais

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

### Tratando os dados

In [2]:
data = pd.read_csv('./data/data.csv')
X = data.drop(columns=['diagnosis', 'Unnamed: 32'])
y = data['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
print(f'X shape: {X.shape}', f'y shape: {y.shape}')

X shape: (569, 31) y shape: (569,)


In [4]:
normalizacao = StandardScaler()
X_train_std = normalizacao.fit_transform(X_train)
X_test_std = normalizacao.transform(X_test)

### Ajustando os hiper-parâmetros

In [5]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [6]:
rfc = RandomForestClassifier()

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "random_state": [42]
}

best_rfc = GridSearchCV(rfc, param_grid, cv=5)
best_rfc.fit(X_train_std, y_train)

print(f'Best Params: {best_rfc.best_params_}')
print(f'Best Score:  {round(best_rfc.best_score_ * 100, 2)}%')

Best Params: {'max_depth': 7, 'n_estimators': 200, 'random_state': 42}
Best Score:  95.6%


In [7]:
abc = AdaBoostClassifier()

param_grid = {
    "learning_rate": [0.1, 0.05, 0.01 ,0.5],
    "n_estimators": [50, 100, 200],
    "random_state": [42]
}

best_abc = GridSearchCV(abc, param_grid, cv=5)
best_abc.fit(X_train_std, y_train)

print(f'Best Params: {best_abc.best_params_}')
print(f'Best Score:  {round(best_abc.best_score_ * 100, 2)}%')

Best Params: {'learning_rate': 0.5, 'n_estimators': 100, 'random_state': 42}
Best Score:  96.7%


### Retreinando os modelos e obtendo as métricas

In [8]:
def calcularMetricas(tn, fp, fn, tp):
    acc =  (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0
    prec =  tp / (tp + fp) if (tp + fp) > 0 else 0
    rec =  tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 =  2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    print(f'Acc: {round(acc * 100, 2)}%')
    print(f'prec: {round(prec * 100, 2)}%')
    print(f'rec: {round(rec * 100, 2)}%')
    print(f'f1: {round(f1 * 100, 2)}%')

In [9]:
best_rfc.fit(X_train_std, y_train)
y_pred_rfc = best_rfc.predict(X_test_std)
best_abc.fit(X_train_std, y_train)
y_pred_abc = best_abc.predict(X_test_std)

In [10]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_rfc).ravel()
calcularMetricas(tn, fp, fn, tp)

Acc: 96.49%
prec: 97.56%
rec: 93.02%
f1: 95.24%


In [11]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_abc).ravel()
calcularMetricas(tn, fp, fn, tp)

Acc: 96.49%
prec: 95.35%
rec: 95.35%
f1: 95.35%


### Conclusão